# Implementing a Neural Network

This is inspired by <https://pub.towardsai.net/building-neural-networks-from-scratch-with-python-code-and-math-in-detail-i-536fae5d7bbf>'s "case study" with corrections.

In [1]:
%matplotlib inline

In [2]:
import time
import io

import numpy
import pandas
import matplotlib

import torch

In [3]:
input_csv = """
person,loss of smell,weight loss,runny nose,body pain,positive?
1,1,0,0,1,1
2,1,0,0,0,1
3,0,0,1,1,0
4,0,1,0,0,0
5,1,1,0,0,1
6,1,0,1,1,1
7,0,0,0,1,0
8,0,0,1,0,0
"""

dataset = pandas.read_csv(io.StringIO(input_csv), index_col="person")
inputs = dataset.iloc[:,:-1].to_numpy().astype('float32')
ground_truth = dataset.iloc[:,-1].to_numpy().reshape(-1, 1).astype('float32')

In [4]:
print(inputs)
print(ground_truth)

[[1. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [1. 1. 0. 0.]
 [1. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]


In [5]:
LEARNING_RATE = 0.05
NUM_ITERATIONS = 10000

## By Hand

In [6]:
def linear(x, weights, bias):
    return numpy.dot(x, weights) + bias

def sigmoid(x):
    return 1.0 / (1.0 + numpy.exp(-x))

def d_sigmoid(x):
    y = sigmoid(x)
    return y * (1.0 - y)

In [7]:
weights = numpy.random.rand(inputs.shape[1], 1)
bias = numpy.random.rand(1)[0]

In [8]:
t0 = time.time()
for i in range(NUM_ITERATIONS):
    y = linear(inputs, weights, bias)
    f = sigmoid(y)
    
    error = numpy.abs(f - ground_truth)
    
    # calculate out partial derivatives for each input
    dE_df = error/(f - ground_truth)
    df_dy = d_sigmoid(y)
    dE_dy = dE_df * df_dy
    dE_dw = numpy.dot(inputs.T, dE_dy)  # dy_dw = x

    # update weights and biases - the error is the sum of error over each input
    weights -= LEARNING_RATE * dE_dw
    bias -= LEARNING_RATE * dE_dy.sum()

    if i % (NUM_ITERATIONS / 10) == 0:
        print("error at step {:5d}: {:10.2e}".format(i, error.sum()))

print("Final weights: {}".format(weights.flatten()))
print("Final bias:    {}".format(bias))
print("{:d} iterations took {:.1f} seconds".format(NUM_ITERATIONS, time.time() - t0))

error at step     0:   3.90e+00
error at step  1000:   1.09e-01
error at step  2000:   5.23e-02
error at step  3000:   3.44e-02
error at step  4000:   2.56e-02
error at step  5000:   2.04e-02
error at step  6000:   1.69e-02
error at step  7000:   1.45e-02
error at step  8000:   1.26e-02
error at step  9000:   1.12e-02
Final weights: [13.46066265 -1.50341607 -1.28100634 -0.93746951]
Final bias:    -5.122024047459534
10000 iterations took 1.7 seconds


In [9]:
predicted_output = sigmoid(linear(inputs, weights, bias))
predicted_output = pandas.DataFrame(
    predicted_output,
    columns=["predicted positive?"],
    index=dataset.index)

output = pandas.concat(
    (dataset, predicted_output),
    axis=1)
output['error'] = output['positive?'] - output['predicted positive?']
output

,loss of smell,weight loss,runny nose,body pain,positive?,predicted positive?,error
person,,,,,,,
1,1,0,0,1,1,0.999390,0.000610
2,1,0,0,0,1,0.999761,0.000239
3,0,0,1,1,0,0.000648,-0.000648
4,0,1,0,0,0,0.001324,-0.001324
5,1,1,0,0,1,0.998926,0.001074
6,1,0,1,1,1,0.997807,0.002193
7,0,0,0,1,0,0.002330,-0.002330
8,0,0,1,0,0,0.001654,-0.001654


## PyTorch

In [10]:
# torch.manual_seed(0)

model = torch.nn.Sequential(
    torch.nn.Linear(inputs.shape[1], 1),
    torch.nn.Sigmoid())

print("Starting weights: {}".format(model[0].weight.flatten()))
print("Starting bias: {}".format(model[0].bias.flatten()))

Starting weights: tensor([-0.2282, -0.0472,  0.4641, -0.4588], grad_fn=<ViewBackward>)
Starting bias: Parameter containing:
tensor([-0.1641], requires_grad=True)


In [11]:
inputs_tensor = torch.from_numpy(inputs)
truth_tensor = torch.from_numpy(ground_truth.reshape(-1, 1))

loss = torch.nn.L1Loss(reduction='sum')

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

model.train()
t0 = time.time()
for i in range(NUM_ITERATIONS):
    f = model(inputs_tensor)

    error = loss(f, truth_tensor)

    optimizer.zero_grad()

    error.backward()

    optimizer.step()

    if i % (NUM_ITERATIONS / 10) == 0:
        print("error at step {:5d}: {:10.2e}".format(i, error.sum()))

print("Final weights: {}".format(next(model.parameters()).detach().numpy().flatten()))
print("Final bias:    {}".format(list(model.parameters())[-1].item()))
print("{:d} iterations took {:.1f} seconds".format(NUM_ITERATIONS, time.time() - t0))

error at step     0:   4.33e+00
error at step  1000:   1.08e-01
error at step  2000:   5.22e-02
error at step  3000:   3.43e-02
error at step  4000:   2.56e-02
error at step  5000:   2.03e-02
error at step  6000:   1.69e-02
error at step  7000:   1.44e-02
error at step  8000:   1.26e-02
error at step  9000:   1.12e-02
Final weights: [13.460027  -1.4970766 -1.2770189 -0.9339364]
Final bias:    -5.12729024887085
10000 iterations took 8.1 seconds


In [12]:
model.eval()

predicted_output = model(inputs_tensor).detach().numpy()
predicted_output = pandas.DataFrame(
    predicted_output,
    columns=["predicted positive?"],
    index=dataset.index)

output = pandas.concat(
    (dataset, predicted_output),
    axis=1)
output['error'] = output['positive?'] - output['predicted positive?']
output

,loss of smell,weight loss,runny nose,body pain,positive?,predicted positive?,error
person,,,,,,,
1,1,0,0,1,1,0.999388,0.000612
2,1,0,0,0,1,0.999759,0.000241
3,0,0,1,1,0,0.000650,-0.000650
4,0,1,0,0,0,0.001326,-0.001326
5,1,1,0,0,1,0.998926,0.001074
6,1,0,1,1,1,0.997810,0.002190
7,0,0,0,1,0,0.002326,-0.002326
8,0,0,1,0,0,0.001652,-0.001652


In [13]:
new_inputs = [
    torch.Tensor([0, 0, 0, 0]),
    torch.Tensor([1, 1, 1, 1]),
    torch.Tensor([1, 1, 1, 0]),
    torch.Tensor([0, 1, 1, 1]),
]
for new_input in new_inputs:
    print("{}: prediction {:.1f}".format(
        ", ".join(["{}={:.0f}".format(*x) for x in zip(dataset.columns[:-1], new_input)]),
        model(new_input).item()))

loss of smell=0, weight loss=0, runny nose=0, body pain=0: prediction 0.0
loss of smell=1, weight loss=1, runny nose=1, body pain=1: prediction 1.0
loss of smell=1, weight loss=1, runny nose=1, body pain=0: prediction 1.0
loss of smell=0, weight loss=1, runny nose=1, body pain=1: prediction 0.0
